In [1]:
from glob import glob
import pandas as pd

In [2]:
xl = pd.ExcelFile('data/P1-PersonalVehicleSalesGlobal.xlsx')

In [53]:
df = xl.parse('pc_sales', header=4)
# get rid of some ugly columns
dftrim = df.iloc[list(range(1,160)), [0] + list(range(4,14))]

# get rid of some crappy rows
dftrim = dftrim.loc[~dftrim['REGIONS/COUNTRIES'].isnull(), :]


# so this data has a bunch of country aggregates and a total in here
# lets add another variable
dftrim['country_scope'] = 'country'


# ok so here are the regions
regions = ["EUROPE", "EU 28 countries + EFTA", "EU 15 countries + EFTA", "EUROPE NEW MEMBERS",
           "RUSSIA, TURKEY & OTHER EUROPE", "AMERICA", "NAFTA", "CENTRAL & SOUTH AMERICA",
           "ASIA/OCEANIA/MIDDLE EAST", "AFRICA"]


dftrim.loc[dftrim['REGIONS/COUNTRIES'].isin(regions), 'country_scope'] = 'region'

# and here is the total
dftrim.loc[dftrim['REGIONS/COUNTRIES'] == "ALL COUNTRIES", 'country_scope'] = 'total'

dftrim

,REGIONS/COUNTRIES,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,country_scope
1,EUROPE,17906455.0,18685556.0,19618588.0,18821599.0,16608761.0,16499863.0,17167600.0,16191359.0,15941854.0,16060143.0,region
2,EU 28 countries + EFTA,15622035.0,15961138.0,16147274.0,14911880.0,14533115.0,13830694.0,13642659.0,12567993.0,12343996.0,13013515.0,region
3,EU 15 countries + EFTA,14565695.0,14820182.0,14842186.0,13602038.0,13668808.0,12984549.0,12815435.0,11773371.0,11554834.0,12113882.0,region
4,AUSTRIA,307915.0,308594.0,298182.0,293697.0,319403.0,328563.0,356145.0,336010.0,319035.0,303318.0,country
5,BELGIUM,480088.0,526141.0,524795.0,535947.0,476194.0,547340.0,572211.0,486737.0,486065.0,482939.0,country
...,...,...,...,...,...,...,...,...,...,...,...,...
154,TUNISIA,25000.0,30000.0,30000.0,33000.0,35000.0,45000.0,35000.0,37000.0,35000.0,37100.0,country
155,UGANDA,3000.0,3500.0,3000.0,3000.0,3000.0,3000.0,3000.0,3000.0,3000.0,3000.0,country
156,ZAMBIA,2300.0,2700.0,2800.0,2600.0,3000.0,2000.0,2000.0,2000.0,2000.0,2000.0,country
157,ZIMBABWE,7000.0,6000.0,3000.0,3000.0,3000.0,2000.0,3000.0,3000.0,3000.0,4100.0,country


In [54]:
# now lets melt the data
df_tidy = dftrim.melt(id_vars=['REGIONS/COUNTRIES', 'country_scope'], var_name='Years', value_name='Sales')
df_tidy.sort_values(by=['country_scope','REGIONS/COUNTRIES', 'Years'], ascending=[False, True, False])

,REGIONS/COUNTRIES,country_scope,Years,Sales
1394,ALL COUNTRIES,total,2013,62857950.0
1084,ALL COUNTRIES,total,2011,57376238.0
929,ALL COUNTRIES,total,2010,55403283.0
774,ALL COUNTRIES,total,2009,49321625.0
619,ALL COUNTRIES,total,2008,49669083.0
...,...,...,...,...
463,ZIMBABWE,country,2007,3000.0
308,ZIMBABWE,country,2006,6000.0
153,ZIMBABWE,country,2005,7000.0
1548,ZIMBABWE,country,2014,4100.0


In [58]:
df2 = pd.read_csv('data/P1-PersonalVehicleSalesGlobal_Clean.csv')

In [68]:
# make lowercase
for col in ['Regions', 'Countries', 'Accuracy']:
        df2[col] = df2[col].str.lower()



In [77]:
df2melt = df2.melt(id_vars=['Regions', 'Countries', 'Accuracy'], var_name='years', value_name='count_cars')
df2melt.sort_values(by=['Regions', 'Countries', 'years'], inplace=True)

df2melt

,Regions,Countries,Accuracy,years,count_cars
116,africa,algeria,measured,2005,"71,046"
260,africa,algeria,measured,2006,"81,803"
404,africa,algeria,measured,2007,"97,239"
548,africa,algeria,measured,2008,"111,753"
692,africa,algeria,measured,2009,"131,259"
...,...,...,...,...,...
761,"russia, turkey & other europe",ukraine,measured,2010,"169,540"
905,"russia, turkey & other europe",ukraine,measured,2011,"207,453"
1049,"russia, turkey & other europe",ukraine,measured,2012,"237,602"
1193,"russia, turkey & other europe",ukraine,measured,2013,"213,322"
